In [1]:
import pandas as pd
import numpy as np

from fuzzywuzzy import process, fuzz

C:\Users\Oscar\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


https://www.statology.org/fuzzy-matching-pandas/#:~:text=Often%20you%20may%20want%20to,function%20from%20the%20difflib%20package.

In [2]:
data = {'name': ['nick', 'david', 'joe', 'ross', 'nicks', 'rossw', 'javier', 'xavier', 'Javi', 'Sergio', 'sergio'],
        'age': ['5', '10', '7', '6', '5', '10', '7', '6', '5', '10', '7']} 
new = pd.DataFrame.from_dict(data)
  
new

,name,age
0,nick,5
1,david,10
2,joe,7
3,ross,6
4,nicks,5
5,rossw,10
6,javier,7
7,xavier,6
8,Javi,5
9,Sergio,10


In [3]:
unique_brand = new['name'].unique().tolist()
sorted(unique_brand)[:20]

['Javi',
 'Sergio',
 'david',
 'javier',
 'joe',
 'nick',
 'nicks',
 'ross',
 'rossw',
 'sergio',
 'xavier']

In [4]:
score_sort = [(x,) + i
             for x in unique_brand 
             for i in process.extract(x, unique_brand,     scorer=fuzz.token_sort_ratio)]

In [5]:
#Create a dataframe from the tuples
similarity_sort = pd.DataFrame(score_sort, columns=['brand_sort','match_sort','score_sort'])
similarity_sort

,brand_sort,match_sort,score_sort
0,nick,nick,100
1,nick,nicks,89
2,nick,Javi,25
3,nick,david,22
4,nick,javier,20
5,david,david,100
6,david,Javi,67
7,david,javier,55
8,david,xavier,55
9,david,nick,22


In [6]:
similarity_sort['sorted_brand_sort'] = np.minimum(similarity_sort['brand_sort'], similarity_sort['match_sort'])
similarity_sort.head()

,brand_sort,match_sort,score_sort,sorted_brand_sort
0,nick,nick,100,nick
1,nick,nicks,89,nick
2,nick,Javi,25,Javi
3,nick,david,22,david
4,nick,javier,20,javier


In [7]:
high_score_sort = similarity_sort[(similarity_sort['score_sort'] >= 80) &
                (similarity_sort['brand_sort'] !=  similarity_sort['match_sort']) &
                (similarity_sort['sorted_brand_sort'] != similarity_sort['match_sort'])]
high_score_sort = high_score_sort.drop('sorted_brand_sort',axis=1).copy()
high_score_sort.head()

,brand_sort,match_sort,score_sort
1,nick,nicks,89
16,ross,rossw,89
31,javier,xavier,83
41,Javi,javier,80
46,Sergio,sergio,100


In [8]:
high_score_sort.groupby(['brand_sort','score_sort']).agg(
                        {'match_sort': ', '.join}).sort_values(
                        ['score_sort'], ascending=False)

,,match_sort
brand_sort,score_sort,
Sergio,100,sergio
nick,89,nicks
ross,89,rossw
javier,83,xavier
Javi,80,javier


In [9]:
similarity_sort

,brand_sort,match_sort,score_sort,sorted_brand_sort
0,nick,nick,100,nick
1,nick,nicks,89,nick
2,nick,Javi,25,Javi
3,nick,david,22,david
4,nick,javier,20,javier
5,david,david,100,david
6,david,Javi,67,Javi
7,david,javier,55,david
8,david,xavier,55,david
9,david,nick,22,david


# Intento funcion

In [68]:
df = {'name': ['nick', 'david', 'joe', 'ross', 'nicks', 'rossw', 'javier', 'xavier', 'Javi', 'Sergio', 'Javer'],
      'age': ['5', '10', '7', '6', '5', '10', '7', '6', '5', '10', '7']} 
df = pd.DataFrame.from_dict(df) 
df['name'] = df['name'].str.title()

In [69]:
unique_brand = df['name'].unique().tolist()

score_sort = [(x,) + i
             for x in unique_brand 
             for i in process.extract(x, unique_brand, scorer=fuzz.token_sort_ratio)]
#Create a dataframe from the tuples
similarity_sort = pd.DataFrame(score_sort, columns=['brand_sort','match_sort','score_sort'])

similarity_sort['sorted_brand_sort'] = np.minimum(similarity_sort['brand_sort'], similarity_sort['match_sort'])


high_score_sort = similarity_sort[(similarity_sort['score_sort'] >= 80) &
                                  (similarity_sort['brand_sort'] !=  similarity_sort['match_sort']) &
                                  (similarity_sort['sorted_brand_sort'] != similarity_sort['match_sort'])]
high_score_sort = high_score_sort.drop('sorted_brand_sort',axis=1).copy()

high_score_sort = high_score_sort.reset_index(drop=True)

high_score_sort

,brand_sort,match_sort,score_sort
0,Nick,Nicks,89
1,Ross,Rossw,89
2,Javier,Xavier,83
3,Javi,Javier,80
4,Javer,Javier,91


In [70]:
high_score_sort = high_score_sort.drop(high_score_sort.index[[3,4]])
high_score_sort

,brand_sort,match_sort,score_sort
0,Nick,Nicks,89
1,Ross,Rossw,89
2,Javier,Xavier,83


In [71]:
df = pd.merge(df, high_score_sort, how='left', left_on='name', right_on='match_sort')
df

,name,age,brand_sort,match_sort,score_sort
0,Nick,5,NaN,NaN,NaN
1,David,10,NaN,NaN,NaN
2,Joe,7,NaN,NaN,NaN
3,Ross,6,NaN,NaN,NaN
4,Nicks,5,Nick,Nicks,89.0
5,Rossw,10,Ross,Rossw,89.0
6,Javier,7,NaN,NaN,NaN
7,Xavier,6,Javier,Xavier,83.0
8,Javi,5,NaN,NaN,NaN
9,Sergio,10,NaN,NaN,NaN


In [72]:
df['name'] = np.where(~df['brand_sort'].isnull(), df['brand_sort'], df['name'])
df

,name,age,brand_sort,match_sort,score_sort
0,Nick,5,NaN,NaN,NaN
1,David,10,NaN,NaN,NaN
2,Joe,7,NaN,NaN,NaN
3,Ross,6,NaN,NaN,NaN
4,Nick,5,Nick,Nicks,89.0
5,Ross,10,Ross,Rossw,89.0
6,Javier,7,NaN,NaN,NaN
7,Javier,6,Javier,Xavier,83.0
8,Javi,5,NaN,NaN,NaN
9,Sergio,10,NaN,NaN,NaN


In [2]:
import difflib 

In [3]:
import pandas as pd

#create two DataFrames
df1 = pd.DataFrame({'team': ['Mavericks', 'Nets', 'Warriors', 'Heat', 'Lakers'],
                    'points': [99, 90, 104, 117, 100]})

df2 = pd.DataFrame({'team': ['Mavricks', 'Warrors', 'Heat', 'Netts', 'Lakes'],
                    'assists': [22, 29, 17, 40, 32]})

In [4]:
#create duplicate column to retain team name from df2
df2['team_match'] = df2['team']

#convert team name in df2 to team name it most closely matches in df1
df2['team'] = df2['team'].apply(lambda x: difflib.get_close_matches(x, df1['team'])[0])

#merge the DataFrames into one
df3 = df1.merge(df2)

#view final DataFrame
print(df3)

        team  points  assists team_match
0  Mavericks      99       22   Mavricks
1       Nets      90       40      Netts
2   Warriors     104       29    Warrors
3       Heat     117       17       Heat
4     Lakers     100       32      Lakes


In [51]:
df_name = {'name': ['Nick', 'David', 'Joe', 'Ross', 'Javier', 'Sergio'],
           'id': ['5', '10', '7', '6', '5', '10']} 
df_name = pd.DataFrame.from_dict(df_name) 
df_name

,name,id
0,Nick,5
1,David,10
2,Joe,7
3,Ross,6
4,Javier,5
5,Sergio,10


In [52]:
df = {'name': ['nick', 'david', 'joe', 'ross', 'nicks', 'rossw', 'javier', 'xavier', 'Javi', 'Sergio', 'Javer'],
      'age': ['5', '10', '7', '6', '5', '10', '7', '6', '5', '10', '7']} 
df = pd.DataFrame.from_dict(df) 
df['name'] = df['name'].str.title()

In [53]:
# df['match'] = df['name']

In [54]:
df['match'] = df['match'].apply(lambda x: difflib.get_close_matches(x, df_name['name'])[0])

KeyError: 'match'

In [55]:
df

,name,age
0,Nick,5
1,David,10
2,Joe,7
3,Ross,6
4,Nicks,5
5,Rossw,10
6,Javier,7
7,Xavier,6
8,Javi,5
9,Sergio,10
